# Task 4
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Add any other imports you need here
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config
from torch.optim import AdamW
from torch.nn import MSELoss

Depending on your approach, you might need to adapt the structure of this template or parts not marked by TODOs.
It is not necessary to completely follow this template. Feel free to add more code and delete any parts that are not required.

In [7]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32  # TODO: Set the batch size according to both training performance and available memory
NUM_EPOCHS = 5  # TODO: Set the number of epochs

train_val = pd.read_csv("train.csv")
test_val = pd.read_csv("test_no_score.csv")

In [17]:
# TODO: Fill out ReviewDataset
class ReviewDataset(Dataset):
    def __init__(self, data_frame, tokenizer=GPT2Tokenizer.from_pretrained('gpt2'), max_length=512):
        self.data_frame = data_frame
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.titles = data_frame['title'].values
        self.sentences = data_frame['sentence'].values
        self.scores = data_frame['score'].values if 'score' in data_frame.columns else None

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, index):
        title = self.titles[index]
        sentence = self.sentences[index]
        text = title + " " + sentence
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        sample = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
        }
        if self.scores is not None:
            score = self.scores[index]
            sample['score'] = torch.tensor(score, dtype=torch.float)
        return sample

In [18]:
train_dataset = ReviewDataset(train_val)
test_dataset = ReviewDataset(test_val)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=16, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=16, pin_memory=True)
# Additional code if needed

d:\Programme\anaconda\envs\IML\lib\site-packages\torch\utils\data\dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
# TODO: Fill out MyModule
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.config = GPT2Config.from_pretrained('gpt2')
        self.regression_head = nn.Linear(self.config.hidden_size, 1)   #Only 1 output value

    def forward(self, sample):
        
        input_ids = sample['input_ids']
        attention_mask = sample['attention_mask']
        
        outputs = self.gpt2(input_ids=input_ids, attention_mask=attention_mask)
        
        hidden_states = outputs.last_hidden_state
        pooled_output = hidden_states[:, -1, :]  # (batch_size, hidden_size)
        score = self.regression_head(pooled_output)
        return score


model = MyModule().to(DEVICE)

d:\Programme\anaconda\envs\IML\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
# TODO: Setup loss function, optimiser, and scheduler

optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = MSELoss()
scheduler = None

model.train()
for epoch in range(NUM_EPOCHS):
    model.train()
    for batch in tqdm(train_loader, total=len(train_loader)):
        batch = batch.to(DEVICE)

        # TODO: Set up training loop
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        scores = batch['score'].to(DEVICE)
        
        # Forward pass
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs.squeeze(), scores)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch [{epoch + 1}/{NUM_EPOCHS}], Loss: {loss.item()}')


  0%|          | 0/391 [00:00<?, ?it/s]

In [ ]:
model.eval()
with torch.no_grad():
    results = []
    for batch in tqdm(test_loader, total=len(test_loader)):
        batch = batch.to(DEVICE)

        # TODO: Set up evaluation loop

    with open("result.txt", "w") as f:
        for val in np.concatenate(results):
            f.write(f"{val}\n")